In [21]:
import csv
import json
import pandas as pd
from datetime import datetime

vendorMaster = {}
with open('/Users/patelrudra/Documents/BOOKS/VendorsMaster.csv', mode ='r')as file:
    masterFile = csv.reader(file)    
    for vendor in masterFile:
        vendorMaster[vendor[1]] = vendor[0]
        
vendorItem = {}
with open('/Users/patelrudra/Documents/BOOKS/Item.csv', mode ='r')as file1:
    masterFile = csv.reader(file1)    
    for vendor in masterFile:
        vendorItem[vendor[1]] = vendor[0]
        
gstConfigurationData = {
    "0" : "1357403000000016427",
    "5" : "1357403000000016433",
    "12" : "1357403000000016439",
    "18" : "1357403000000016445",
    "28" : "1357403000000016451"
}
        
with open('/Users/patelrudra/Documents/MedKart/wms/Purchase May-2023.csv', mode ='r')as file:
    csvFile = csv.reader(file)
    data_list = []
    null_voucher_dict = []
    data={}
    count=0
    Bill=True
    notItems=[]
    str_line=['TAX6586', '23SZ011097', '23SZ011428', '23SZ011804', 'TAX7310', 'DSS2322634', 'DSS2322635', 'TAX7308', '23SZ012159', '23SZ012189', '24T010101', '23SZ012484', '23SZ012486', '23SZ002765', '23SZ013343', '24T010725', '24T011030', '23SZ014228', '23SZ208272', '111239', '111238', '111586', '111585', '111581', '111860', '111782', '111781', '111778', '111861', '112145', '112144', '112366', '112428', '112370', '112365', '23SZ014835', '112640', '112639', '23SZ209472', '23SZ209498', '23SZ209653', '23SZ209611', '112913', '112906', '23SZ015052', '112907', '112912', '113237', '113238', '113255', '113252', '113253', '113239', '23SZ015768', '23SZ015890', '113522', '113534', '113529', '113530', '113531', '113789', '23SZ016063', '113787', '113788', '113781', '113782', '114066', '114067', '114065', '114078', '114079', '23SZ016599', '114320', '114327', '114328', '114319', '114314', '114326', '114573', '114572', '114574', '114846', '114845', '24T015816', '115111', '115112', '115106', '115110']
    
    for entry in csvFile:
        if entry[4] == '':
            if entry[5] in str_line:
                if entry[1] == 'Grand Total':
                    continue;
                data1 = {
                    'item_id': vendorItem[entry[1]],
                    'rate':entry[8],
                    "itc_eligibility":"eligible",
                    "quantity":"1",
                    "account_id":"1357403000000000567",
                    "item_order":len(data_list)+1,
                    "tax_id" : gstConfigurationData[entry[1].split("_")[1]]
                 }

                data_list.append(data1)
        else:
            if entry[5] in str_line:
                data['line_items'] = data_list
                null_voucher_dict.append(data)
                data_list=[]
                date_object = datetime.strptime(entry[0], "%d-%b-%y")
                formatted_date = date_object.strftime("%Y-%m-%d")
                if entry[20]=="":
                    entry[20]=0
                if entry[16]=="":
                    entry[16]=0

                data = {
                    'date': formatted_date,
                    'vendor_id': vendorMaster[entry[1]],
                    'supplier Name': entry[2],
                    'bill_number': entry[5],
                    'source_of_supply':"24",
                    "destination_of_supply":"24",
                    "adjustment" : float(entry[16])+float(entry[20])
                 }
            
    for data in null_voucher_dict:
        json_data = json.dumps(data, indent=4)
        with open("sample.json", "a") as outfile:
            outfile.write(json_data + '\n')  

In [22]:
len(null_voucher_dict)

89

In [19]:
import requests
import json
url = "https://www.zohoapis.in/books/v3/bills?organization_id=60022053195"
headers = {
  'Content-Type': 'application/json',
  'Authorization': 'Zoho-oauthtoken 1000.a756c019439a696ac27f03d5c79dfdea.e669c43b75b82d513a89109ddcc231fc'
}
failedData = []
for data in null_voucher_dict:
    payload = json.dumps(data)


    response = requests.request("POST", url, headers=headers, data=payload)
    if response.status_code >= 200 and response.status_code < 300:  # Assuming a successful response has a status code of 200
        data = response.json()
        # Process the response data as needed
        # print(data)
    else:

        print(f"API request failed with status code {response.status_code}")
        response_content = response.content.decode('utf-8')
        failedData.append(data['bill_number'])
        print(response_content)

print("__________________________________")
print("Finished")
print("__________________________________")
print("failed records bill number : ")
print(failedData)

API request failed with status code 400
{"code":13011,"message":"A bill with this number has already been created for this vendor. Please check and try again."}
__________________________________
Finished
__________________________________
failed records bill number : 
['23SZ012486']
